In [ ]:
# LangChain Tools Overview:
# - Tools allow LLMs to interact with external sources like search engines, databases, and APIs.
# - Common Built-in Tools:
#   - SerpAPIWrapper → Fetches real-time Google search results.
#   - ArxivAPIWrapper → Retrieves latest research papers from ArXiv.
#   - WikipediaAPIWrapper → Fetches Wikipedia articles for general knowledge.
# - Custom tools can be created using @tool decorator or Tool() class.
# - Tools can be integrated into an agent to allow dynamic selection based on user queries.


In [ ]:
from langchain_community.tools import WikipediaQueryRun 
from langchain_community.utilities import WikipediaAPIWrapper 

In [ ]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=150)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
wiki.name

In [ ]:
from langchain_community.document_loaders import  WebBaseLoader
from langchain_community.vectorstores import FAISS 
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_community.embeddings import OllamaEmbeddings

embeddings=OllamaEmbeddings(model="llama3.2")
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever 

In [ ]:
from langchain.tools.retriever import create_retriever_tool 
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [ ]:
retriever_tool.name 

In [ ]:
#Arxiv tool 

from langchain_community.utilities import ArxivAPIWrapper 
from langchain_community.tools import ArxivQueryRun 

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

In [ ]:
# MultiSearch Agent RAG Overview:
# - Uses multiple search tools (Wikipedia, ArXiv, Retriever Tool) to fetch relevant data.
# - WikipediaAPIWrapper → Retrieves general knowledge from Wikipedia.
# - ArxivAPIWrapper → Fetches latest research papers from ArXiv.
# - Retriever Tool → Retrieves domain-specific documents from a vector database (e.g., LanceDB).
# - LangChain Agent dynamically selects the best tool based on query type.
# - Enhances LLM responses with real-world, research-based, and domain-specific knowledge.
# - Helps reduce hallucinations and improves answer accuracy in RAG-based systems.


In [ ]:
tools=[wiki,arxiv,retriever_tool]

In [ ]:
tools

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
from langchain import hub 
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


In [ ]:
# LangChain Agents Overview:
# - Agents dynamically decide which tool to use based on the query.
# - They analyze input, select the right tool (e.g., Wikipedia, ArXiv, Retriever), and process results.
# - Useful for multi-search RAG systems where different sources provide diverse information.
# - Reduce hallucinations by retrieving real-time or factual data before generating responses.
# - `AgentExecutor` runs the agent, allowing it to call tools and return enriched answers.
# - Works well with LLMs (e.g., OpenAI, Ollama's Gemma 3B) to improve response accuracy.


In [ ]:
from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm, tools, prompt)


In [ ]:
##Agent  executor
from langchain.agents import AgentExecutor 

agent_executor= AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor 

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})